In [4]:
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
# print(plotly.__version__)

In [5]:
# import Excel file from Windows
# fileDir = r'C:\Users\HarleyKing\OneDrive - LuminUltra Technologies Ltd\Documents\Python Scripts\canonical_pcr_data'
fileDir = r'qPCR_data'
# fileName = r'20220708_122234_SRB_dsr_test_2.xls' #cy5 data
# fileName = r'Exp800.05.create.primer.matrix.micH.xlsx' #fam data
# fileName = r'AF Fuel Primer Matrix Run 2 01Jun21 GH.xls' #fam data
fileName = r'three_fluor_assay.xls' #fam data

# create dataframe
dfExcel = pd.read_excel(fileDir+'\\'+fileName, sheet_name=None, header=None) #get all sheets

# show all sheet names
sheetNames = list(dfExcel.keys())
print(sheetNames)

['Detectors', 'Sample', 'Plate', 'Program', 'Quan. Raw Data', 'Quan. Result', 'Quan. AmpData-FAM', 'Quan. AmpData-HEX', 'Quan. AmpData-Cy5']


In [6]:
class Fluor:
    probe: str
    def __init__(self, probe):
        self.probe = probe
        # self.maxRow = self.getMaxRow()
    def dfCreate(self): # make a df from the excel data
        # find probe in sheet names
        probe_sheet = [sheet for sheet in sheetNames if self.probe in sheet]
        # create dataframe from sheet name
        df = dfExcel[probe_sheet[0]]
        # assign first row to be column header
        df.columns = df.iloc[0]
        # drop the first row (duplicate of header)
        df = df.drop(df.index[0])
        # convert all columns to numberic
        df = df.apply(pd.to_numeric, errors='coerce')
        # make index the first column
        df.set_index('Cycle', inplace=True)
        # fluor values less than 1 get 0
        df = df.where(df>=1, 0)
        return df
    def getMaxRow(self):
        df = self.dfCreate()
        # get max value for each row and convert to list
        maxRow = df.loc[df.index == 40].values.flatten().tolist()
        # round every element in list to one decimal place, convert to str
        maxRowRound = [str(round(x,1)) for x in maxRow]
        return maxRowRound
    def plotAllWells(self):
        df = self.dfCreate()
        # make plot of wells
        fig = px.line(
            df,
            x=df.index,
            y=list(df.columns),
            color_discrete_sequence=px.colors.qualitative.Set3)
        fig.update_layout(
            height=500,
            width=1000,
            xaxis_title='Cycle',
            yaxis_title='RFU ({})'.format(self.probe),
            title_text='Cycle vs Fluorescence ({})'.format(self.probe),
            legend_title='Well')
        return fig.show()
    def plotIndWells(self):
        df = self.dfCreate() 
        maxRow = self.getMaxRow()
        # make subplots of individual wells
        colorsList = px.colors.qualitative.Set3
        figSub = make_subplots(
            rows=8, cols=12,
            shared_xaxes=True,
            shared_yaxes='all',
            start_cell='top-left',
            subplot_titles=list(df.columns)) #, subplot_titles=sub_titles, print_grid=False)
        for r in range (0,8): #loop through rows
            for c in range(0,12):
                figSub.add_trace(go.Scatter(
                    x=df.index,
                    y=df.iloc[:,r+(r*11)+c], # this pattern translates r=8, c=12 to 0..95 columns in df (0 indexed)
                    name=df.columns[r+(r*11)+c], # gets column name eg 'A01'
                    mode="lines+text", # allows for text to be inserted onto ind plot
                    line_color=colorsList[c], # assign color to each well consistent with all-well plot
                    text= ['']*32 + [maxRow[r+(r*11)+c]]+['']*7, # inserts max value onto plot at 33rd point; looks nice here
                    textposition='top left',
                    textfont=dict(
                        family="sans serif",
                        size=8,
                        color="black")), # want to stand out
                    row=r+1,
                    col=c+1)
        figSub.update_layout(
            height=800,
            width=1000,
            showlegend=False,
            title_text='Cycle vs Fluorescence ({})'.format(self.probe))
        return figSub.show()
                


In [7]:
# instantiate probes
probeCy5 = Fluor('Cy5')
probeFAM = Fluor('FAM')
probeHEX = Fluor('HEX')
probeSYBR = Fluor('SYBR')

Fluor.plotAllWells(probeFAM)
Fluor.plotIndWells(probeFAM)
